In [ ]:
!nvidia-smi

In [ ]:
# stahneme si data
!wget https://graphicwg.irafm.osu.cz/storage/retinopathy.zip --no-check-certificate
!unzip -q retinopathy.zip
!pip install -q -U albumentations
!pip install -q -U focal-loss

In [3]:
import albumentations as A
import focal_loss

# naimportujeme vsechny knihovny
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime, os

# logovani 
%load_ext tensorboard

In [4]:
#nacteme csv kde jsou informace o tridach 
df = pd.read_csv("trainLabels.csv")

In [5]:
# https://albumentations.ai/docs/api_reference/augmentations/transforms/
aug_str = "rotate_and_blur"
def augument(img):
  t = A.Compose([
                  A.CLAHE(),
                  A.OneOf([
                    A.Blur(blur_limit=3, p=0.5),
                    A.OpticalDistortion(),
                  ], p=0.5),
                  A.OneOf([
                      A.IAASharpen(p=0.5),
                      A.RandomBrightnessContrast(),
                  ], p=0.4)
  ]
  )
  return t(image=img.astype(np.uint8))['image'].astype(np.float32)

In [ ]:
# vytvorime strukturu pro nacitani dat a definujeme zpusob nacitani
# https://pypi.org/project/split-folders/ -> python knihovna pro split datasetu v slozce
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html split datasetu v pameti

datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=augument, rescale=1./255., validation_split=.33)
datagen_valid = tf.keras.preprocessing.image.ImageDataGenerator(                                 rescale=1./255., validation_split=.33)
seed = 13

train_generator = datagen_train.flow_from_dataframe(
    dataframe = df,
    x_col = "image",
    y_col = "level",
    directory = "train_extracted/",
    target_size = (224,224),
    class_mode = "raw",
    seed = seed,
    subset = "training"
)

valid_generator = datagen_valid.flow_from_dataframe(
    dataframe = df,
    x_col = "image",
    y_col = "level",
    directory = "train_extracted/",
    target_size = (224,224),
    class_mode = "raw",
    subset = "validation"
)

In [ ]:
imgs, labels = train_generator.next()

plt.figure(figsize=(10,10))
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.title(labels[i])
  plt.imshow(imgs[i])
plt.show()

In [11]:
# pripravime si model a dame to trenovat
# logovaci adresar
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_"+aug_str)

# https://keras.io/api/applications/
# https://www.kaggle.com/manifoldix/inceptionv3-for-retinopathy-gpu-hr
# https://pypi.org/project/focal-loss/
model =  tf.keras.applications.efficientnet.EfficientNetB0()
model.compile(loss=focal_loss.SparseCategoricalFocalLoss(gamma=2),
              # loss='sparse_categorical_crossentropy',
              optimizer='Adam', metrics=['sparse_categorical_accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, update_freq='batch')
model.fit(train_generator, 
          validation_data=valid_generator,
          epochs=10,
          callbacks=[tensorboard_callback],
          steps_per_epoch=20,
          workers=4,
          )

Epoch 1/10
20/20 [==============================] - 40s 2s/step - loss: 4.0407 - sparse_categorical_accuracy: 0.4029 - val_loss: 15.9518 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/10
20/20 [==============================] - 27s 1s/step - loss: 0.7638 - sparse_categorical_accuracy: 0.5764 - val_loss: 3.0186 - val_sparse_categorical_accuracy: 0.7266
Epoch 3/10
20/20 [==============================] - 27s 1s/step - loss: 0.5331 - sparse_categorical_accuracy: 0.7210 - val_loss: 0.6401 - val_sparse_categorical_accuracy: 0.7266
Epoch 4/10
20/20 [==============================] - 27s 1s/step - loss: 0.4770 - sparse_categorical_accuracy: 0.7333 - val_loss: 0.4962 - val_sparse_categorical_accuracy: 0.7266
Epoch 5/10
20/20 [==============================] - 27s 1s/step - loss: 0.4761 - sparse_categorical_accuracy: 0.7036 - val_loss: 0.4941 - val_sparse_categorical_accuracy: 0.7266
Epoch 6/10
20/20 [==============================] - 27s 1s/step - loss: 0.5573 - sparse_categorical_accur

In [10]:
%tensorboard --logdir logs

<IPython.core.display.Javascript object>

In [ ]:
!rm -rf logs/*

In [ ]:
imgs, labels = train_generator.next()
preds = model.predict(imgs)

plt.imshow(imgs[0].astype(np.float32))
plt.show()
print('label je: ', labels[0])
print('predikce je: ', np.argmax(preds[0]))